In [1]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (188 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [2]:
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [3]:
!which pdfinfo

/usr/bin/pdfinfo


In [4]:
!pip install --upgrade nltk

In [5]:
import nltk

# Download the punkt tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
!pip install langchain-community
!pip install langchain-chroma
!pip install sentence_transformers
!pip install langchain-google-genai
!pip install "unstructured[all-docs]"
!pip install langchain-google-genai
!pip install pytesseract

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from unstructured.partition.pdf import partition_pdf
import pytesseract
import os
from langchain.schema.document import Document
import uuid

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-8-fa4e1dacca45>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [9]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('Google_API_Key')

In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

In [11]:
pdf_path = "/content/drive/MyDrive/economy_Doc_merged.pdf"
image_output_path = "/content/image_output_path"
output_path = "/content/output"

In [12]:
# Get pdf elements
raw_pdf_elements = partition_pdf(
    filename=pdf_path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=1600,
    new_after_n_chars=1140,
    combine_text_under_n_chars=800,
    image_output_dir_path=image_output_path,
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
# Extract texts and tables
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [14]:
print(len(tables), len(texts))

1 9


In [15]:
tables[0]

'Period Exports Imports Trade Balance Trade balance as % of GDP 1991-92 32,553 43,198 -10,645 2.1 1996-97 118,817 138,920 -20,103 1.6 1997-98 130,100 154,176 -24,076 1.7 1998-99 139,753 178,332 -38,580 2.4 1999-00 159,561 215,236 -55,675 3.1 2000-01 203,571 230,873 -27,302 1.4 2001-02 209,018 245,200 -36,181 17 2002-03 255,137 297,206 -42,069 1.8 2003-04 293,367 359,108 -65,741 2.6 2004-05 375,340 501,065 -125,725 44 2005-06 456,418 660,409 -203,991 6.2 2006-07 571,779 840,506 -268,727 71 20007-08 655,864 1,0123,12 -356448 7.8 20008-09 840,755 1,374,436 -533,680 10.1 2009-10 845,534 1,363,736 -518,202 8.5'

In [16]:
texts[0]

'IOSR Journal of Economics and Finance (IOSR-JEF) e-ISSN: 2321-5933, p-ISSN: 2321-5925.Volume 3, Issue 3. (Mar-Apr. 2014), PP 19-24 www.iosrjournals.org\n\nAn Overview of Indian Economy (1991-2013)\n\nNamrata Anand\n\nResearch Scholar Department of Economics, Dayalbagh Educational Institute (Deemed University)\n\nAbstract: Since 1951, India has fully-fledged as a planned economy. The first few plans focused on growth with strengthening of the manufacturing and industrial sector to form the backbone of the Indian economy. Other principal areas of planning were agriculture, poverty alleviation, employment generation, social development etc. Back in 1991, India saw itself battling its most critical economic and currency crisis ever, but after economic reforms and adopting the policy of LPG (Liberalization, Privatization, and Globalization) Indian economy performed well. Then again due to global financial crisis in 2008 Indian economy again interrupted and going through another turbulent p

In [17]:
image_path="/content/figures"

In [18]:
# Extract texts from images
text_from_images = []

for img_file in os.listdir(image_path):
    if img_file.endswith('.jpg'):
      full_path = os.path.join(image_path, img_file)
      text = pytesseract.image_to_string(full_path)
      text_from_images.append(text)

In [19]:
print(len(text_from_images))

1


In [20]:
print(text_from_images[0])

Artificial Intelligence

Machine Learning.

 

Generative Al

It learns from existing data &
generates brand new artifacts.
Gen Al produces new content:
images, speech, video,
formulas, music, text, code.
Risks include: bias, accuracy,
transparency, cybersecurity.
In 2023 Gen Al is used in:
marketing, sales, gaming,
graphic design, programming,
finance, healthcare.

Op DEVPRO



In [21]:
# Create Document objects from texts, tables, and text_from_images
id_key = "document_id"

doc_ids = [str(uuid.uuid4()) for _ in texts]
table_ids = [str(uuid.uuid4()) for _ in tables]
image_ids = [str(uuid.uuid4()) for _ in text_from_images]

loader = [
    Document(metadata={id_key: doc_ids[i]}, page_content=c)
    for i, c in enumerate(texts)
] + [
    Document(metadata={id_key: table_ids[i]}, page_content=c)
    for i, c in enumerate(tables)
] + [
    Document(metadata={id_key: image_ids[i]}, page_content=c)
    for i, c in enumerate(text_from_images)
]

In [22]:
# Create a Chroma vector store
chroma = Chroma.from_documents(
    documents = loader,
    embedding = embeddings,
    persist_directory=output_path
)

In [23]:
query = "Give an overview of Indian Economy?"
docs = chroma.similarity_search(query)
print(docs)

[Document(metadata={'document_id': '0e9873ac-fa91-411f-9a4e-33e0d82e54cd'}, page_content='IOSR Journal of Economics and Finance (IOSR-JEF) e-ISSN: 2321-5933, p-ISSN: 2321-5925.Volume 3, Issue 3. (Mar-Apr. 2014), PP 19-24 www.iosrjournals.org\n\nAn Overview of Indian Economy (1991-2013)\n\nNamrata Anand\n\nResearch Scholar Department of Economics, Dayalbagh Educational Institute (Deemed University)\n\nAbstract: Since 1951, India has fully-fledged as a planned economy. The first few plans focused on growth with strengthening of the manufacturing and industrial sector to form the backbone of the Indian economy. Other principal areas of planning were agriculture, poverty alleviation, employment generation, social development etc. Back in 1991, India saw itself battling its most critical economic and currency crisis ever, but after economic reforms and adopting the policy of LPG (Liberalization, Privatization, and Globalization) Indian economy performed well. Then again due to global financ

In [24]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=chroma.as_retriever())

In [25]:
query = "What is generative ai?"

In [26]:
response = qa.run(query)
print(response)

<ipython-input-26-4431f097a381>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = qa.run(query)


Generative AI is a type of artificial intelligence that learns from existing data and then creates brand new artifacts. It can produce a variety of content, including:

* **Images**
* **Speech**
* **Video**
* **Formulas**
* **Music**
* **Text**
* **Code**

While generative AI has many potential benefits, there are also risks to consider, such as:

* **Bias:** The AI may reflect biases present in the data it was trained on.
* **Accuracy:** The generated content may not always be accurate or truthful.
* **Transparency:** It can be difficult to understand how the AI generates its outputs.
* **Cybersecurity:**  Generative AI could be used to create malicious content or for other cybersecurity threats.

In 2023, generative AI is being used in a variety of fields, including:

* **Marketing**
* **Sales**
* **Gaming**
* **Graphic design**
* **Programming**
* **Finance**
* **Healthcare** 



In [29]:
# Prompt for Question and Answering
import sys
while True:
    user_input = input(f"Input Prompt: ")
    if user_input == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        continue
    result = qa({'query': user_input})
    print(f"Answer: {result['result']}")

Input Prompt: what is generative ai?


<ipython-input-29-242f5478539f>:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({'query': user_input})


Answer: Generative AI is a type of artificial intelligence that learns from existing data and then creates brand new artifacts.  It can produce things like:

* Images
* Speech
* Video
* Formulas
* Music
* Text
* Code

Some potential risks of generative AI include:

* **Bias:**  The AI might reflect biases present in the data it was trained on.
* **Accuracy:** The generated content might not always be accurate or truthful.
* **Transparency:** It can be difficult to understand how the AI makes its decisions.
* **Cybersecurity:**  There are concerns about the potential for misuse of generative AI.

In 2023, generative AI is being used in a variety of fields, including:

* Marketing
* Sales
* Gaming
* Graphic Design
* Programming
* Finance
* Healthcare 

Input Prompt: what is indian economy?
Answer: The provided text focuses on the Indian economy from 1991 to 2013, particularly highlighting the impact of economic reforms and the global financial crisis. 

Here's a summary of what the text 

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
